In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')

In [3]:

df = pd.read_csv("../data/cleaned_insurance_data.csv")
df.columns = [c.strip() for c in df.columns]
df.head()


,UnderwrittenCoverID,PolicyID,PostalCode,mmcode,RegistrationYear,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,CustomValueEstimate,...,Product,StatutoryClass,StatutoryRiskType,Citizenship,Bank,AccountType,MaritalStatus,Gender,LegalType,Title
0,145249.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.0,...,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,,First National Bank,Current account,Not specified,Not specified,Close Corporation,Mr
1,145249.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.0,...,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,,First National Bank,Current account,Not specified,Not specified,Close Corporation,Mr
2,145255.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.0,...,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,,First National Bank,Current account,Not specified,Not specified,Close Corporation,Mr
3,145247.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.0,...,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,,First National Bank,Current account,Not specified,Not specified,Close Corporation,Mr
4,145247.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.0,...,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,,First National Bank,Current account,Not specified,Not specified,Close Corporation,Mr


In [4]:
df['has_claim'] = df['TotalClaims'] > 0
df['severity_per_claim'] = df['TotalClaims'].where(df['TotalClaims']>0, np.nan)
df['margin'] = df['TotalPremium'] - df['TotalClaims']

df[['PolicyID','Province','PostalCode','Gender','has_claim','severity_per_claim','margin']].head()


,PolicyID,Province,PostalCode,Gender,has_claim,severity_per_claim,margin
0,12827.0,Gauteng,1459.0,Not specified,False,NaN,21.929825
1,12827.0,Gauteng,1459.0,Not specified,False,NaN,21.929825
2,12827.0,Gauteng,1459.0,Not specified,False,NaN,512.848070
3,12827.0,Gauteng,1459.0,Not specified,False,NaN,3.256435
4,12827.0,Gauteng,1459.0,Not specified,False,NaN,50.474737


In [5]:
df_policy = df.groupby('PolicyID').agg({
    'TotalClaims': 'sum',
    'TotalPremium': 'sum',
})


In [6]:
demo_cols = ['Province', 'Gender', 'PostalCode']
df_demo = df.groupby('PolicyID')[demo_cols].first()


In [7]:
df_clean = df_policy.merge(df_demo, on='PolicyID')


In [8]:
df_clean['has_claim'] = df_clean['TotalClaims'] > 0
df_clean['severity_per_claim'] = df_clean['TotalClaims'].where(df_clean['has_claim'], np.nan)
df_clean['margin'] = df_clean['TotalPremium'] - df_clean['TotalClaims']


In [9]:
df_clean.head()


,TotalClaims,TotalPremium,Province,Gender,PostalCode,has_claim,severity_per_claim,margin
PolicyID,,,,,,,,
14.0,0.0,915.785877,Western Cape,Male,7530.0,False,NaN,915.785877
15.0,0.0,151.867402,Western Cape,Female,7405.0,False,NaN,151.867402
17.0,0.0,1692.981018,Western Cape,Male,7785.0,False,NaN,1692.981018
19.0,0.0,1161.045201,Western Cape,Female,7439.0,False,NaN,1161.045201
20.0,0.0,1671.049261,Western Cape,Male,7750.0,False,NaN,1671.049261


In [12]:

n_before = len(df_clean)
mask_ns = df_clean['Gender'].astype(str).str.strip().str.upper() == "NOT SPECIFIED"

n_dropped = mask_ns.sum()

df_clean = df_clean.loc[~mask_ns].copy()

n_after = len(df_clean)

print(f"Dropped {n_dropped} rows where Gender == 'Not specified' (from {n_before} → {n_after})")


df_clean['gender_clean'] = (
    df_clean['Gender']
    .astype(str)
    .str.strip()
    .str.upper()
    .replace({
        'FEMALE': 'F',
        'MALE': 'M'
    })
)

print("\nUnique cleaned gender values:", df_clean['gender_clean'].unique())


Dropped 0 rows where Gender == 'Not specified' (from 300 → 300)

Unique cleaned gender values: ['M' 'F' 'UNKNOWN']


In [13]:
df.head()

,UnderwrittenCoverID,PolicyID,PostalCode,mmcode,RegistrationYear,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,CustomValueEstimate,...,Citizenship,Bank,AccountType,MaritalStatus,Gender,LegalType,Title,has_claim,severity_per_claim,margin
0,145249.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.0,...,,First National Bank,Current account,Not specified,Not specified,Close Corporation,Mr,False,NaN,21.929825
1,145249.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.0,...,,First National Bank,Current account,Not specified,Not specified,Close Corporation,Mr,False,NaN,21.929825
2,145255.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.0,...,,First National Bank,Current account,Not specified,Not specified,Close Corporation,Mr,False,NaN,512.848070
3,145247.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.0,...,,First National Bank,Current account,Not specified,Not specified,Close Corporation,Mr,False,NaN,3.256435
4,145247.0,12827.0,1459.0,44069150.0,2004.0,6.0,2597.0,130.0,4.0,119300.0,...,,First National Bank,Current account,Not specified,Not specified,Close Corporation,Mr,False,NaN,50.474737


In [14]:
df_gender = df_clean[df_clean['gender_clean'].isin(['M','F'])].copy()

print(df_gender['gender_clean'].unique())
print(df_gender.shape)


['M' 'F']
(172, 9)


In [15]:
df_clean.head()

,TotalClaims,TotalPremium,Province,Gender,PostalCode,has_claim,severity_per_claim,margin,gender_clean
PolicyID,,,,,,,,,
14.0,0.0,915.785877,Western Cape,Male,7530.0,False,NaN,915.785877,M
15.0,0.0,151.867402,Western Cape,Female,7405.0,False,NaN,151.867402,F
17.0,0.0,1692.981018,Western Cape,Male,7785.0,False,NaN,1692.981018,M
19.0,0.0,1161.045201,Western Cape,Female,7439.0,False,NaN,1161.045201,F
20.0,0.0,1671.049261,Western Cape,Male,7750.0,False,NaN,1671.049261,M


In [16]:
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")


ALPHA = 0.05
TOP_N_ZIPS = 10  
MIN_COUNT_FOR_PAIRWISE = 10 


In [17]:

print("df_clean shape:", df_clean.shape)
display(df_clean.head())


required = ['TotalClaims','TotalPremium','Province','PostalCode','gender_clean','has_claim','severity_per_claim','margin']
missing = [c for c in required if c not in df_clean.columns]
if missing:
    raise RuntimeError("Missing required columns in df_clean: " + ", ".join(missing))


df_clean shape: (300, 9)


,TotalClaims,TotalPremium,Province,Gender,PostalCode,has_claim,severity_per_claim,margin,gender_clean
PolicyID,,,,,,,,,
14.0,0.0,915.785877,Western Cape,Male,7530.0,False,NaN,915.785877,M
15.0,0.0,151.867402,Western Cape,Female,7405.0,False,NaN,151.867402,F
17.0,0.0,1692.981018,Western Cape,Male,7785.0,False,NaN,1692.981018,M
19.0,0.0,1161.045201,Western Cape,Female,7439.0,False,NaN,1161.045201,F
20.0,0.0,1671.049261,Western Cape,Male,7750.0,False,NaN,1671.049261,M


In [18]:

n_total = len(df_clean)
claim_freq = df_clean['has_claim'].mean()
severity_mean = df_clean['severity_per_claim'].dropna().mean()
avg_margin = df_clean['margin'].mean()
loss_ratio = df_clean['TotalClaims'].sum() / df_clean['TotalPremium'].sum()

print(f"Policies: {n_total}")
print(f"Overall claim frequency: {claim_freq:.4f}")
print(f"Average severity (given claim): {severity_mean:.2f}")
print(f"Average margin per policy: {avg_margin:.2f}")
print(f"Overall loss ratio = TotalClaims / TotalPremium: {loss_ratio:.4f}")


Policies: 300
Overall claim frequency: 0.0167
Average severity (given claim): 1025.44
Average margin per policy: 8994.80
Overall loss ratio = TotalClaims / TotalPremium: 0.0019


In [19]:
def kpi_summary(group):
    n = len(group)
    freq = group['has_claim'].mean()
    severity = group['severity_per_claim'].dropna().mean()
    margin = group['margin'].mean()
    loss_ratio = group['TotalClaims'].sum() / group['TotalPremium'].sum() if group['TotalPremium'].sum() != 0 else np.nan
    return pd.Series({'n':n,'claim_freq':freq,'severity':severity,'mean_margin':margin,'loss_ratio':loss_ratio})

def diff_in_proportions(p1, p2):
    return p1 - p2

def mean_diff(a, b):
    return np.nanmean(a) - np.nanmean(b)


In [20]:

prov_table = df_clean.groupby('Province').apply(kpi_summary).sort_values('n', ascending=False)
display(prov_table)

cont_prov = pd.crosstab(df_clean['Province'], df_clean['has_claim'])
chi2_p, p_chi_p, dof, expected = stats.chi2_contingency(cont_prov)
print("Claim freq chi-square: chi2=%.3f, p=%.5f" % (chi2_p, p_chi_p))


severity_groups = [g['severity_per_claim'].dropna().values for name,g in df_clean.groupby('Province')]

severity_groups = [g for g in severity_groups if len(g)>0]
if len(severity_groups) > 1:
    kw_s, p_kw_s = stats.kruskal(*severity_groups)
    print("Severity Kruskal-Wallis: H=%.3f, p=%.5f" % (kw_s, p_kw_s))
else:
    print("Not enough severity data across provinces for Kruskal-Wallis.")

margin_groups = [g['margin'].dropna().values for name,g in df_clean.groupby('Province')]
margin_groups = [g for g in margin_groups if len(g)>0]
if len(margin_groups) > 1:
    kw_m, p_kw_m = stats.kruskal(*margin_groups)
    print("Margin Kruskal-Wallis: H=%.3f, p=%.5f" % (kw_m, p_kw_m))
else:
    print("Not enough margin data across provinces for Kruskal-Wallis.")


C:\Users\hp\AppData\Local\Temp\ipykernel_12536\1003705654.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  prov_table = df_clean.groupby('Province').apply(kpi_summary).sort_values('n', ascending=False)


,n,claim_freq,severity,mean_margin,loss_ratio
Province,,,,,
Gauteng,183.0,0.016393,1142.397661,8347.622552,0.002238
Western Cape,77.0,0.000000,NaN,6466.083949,0.000000
KwaZulu-Natal,20.0,0.100000,850.000000,16552.264879,0.005109
North West,10.0,0.000000,NaN,21399.667134,0.000000
Limpopo,7.0,0.000000,NaN,12899.045784,0.000000
Mpumalanga,2.0,0.000000,NaN,18779.315044,0.000000
Northern Cape,1.0,0.000000,NaN,43.859649,0.000000


Claim freq chi-square: chi2=10.119, p=0.11971
Severity Kruskal-Wallis: H=0.667, p=0.41422
Margin Kruskal-Wallis: H=52.331, p=0.00000


In [21]:
from itertools import combinations
prov_names = df_clean['Province'].unique()
pair_results = []
for a,b in combinations(prov_names, 2):
    arr_a = df_clean.loc[df_clean['Province']==a,'severity_per_claim'].dropna()
    arr_b = df_clean.loc[df_clean['Province']==b,'severity_per_claim'].dropna()
    if len(arr_a) >= MIN_COUNT_FOR_PAIRWISE and len(arr_b) >= MIN_COUNT_FOR_PAIRWISE:
        t, p = stats.ttest_ind(arr_a, arr_b, equal_var=False, nan_policy='omit')
        pair_results.append({'A':a,'B':b,'nA':len(arr_a),'nB':len(arr_b),'p':p,
                             'meanA':arr_a.mean(),'meanB':arr_b.mean(),
                             'mean_diff':arr_a.mean()-arr_b.mean()})
pair_df = pd.DataFrame(pair_results)
if not pair_df.empty:
    rej, p_adj, _, _ = multipletests(pair_df['p'].values, alpha=ALPHA, method='fdr_bh')
    pair_df['p_adj'] = p_adj
    pair_df['reject'] = rej
    display(pair_df.sort_values('p_adj').reset_index(drop=True))
else:
    print("No pairwise comparisons meet the minimum count thresholds.")


No pairwise comparisons meet the minimum count thresholds.


In [23]:
top_zips = df_clean['PostalCode'].value_counts().nlargest(TOP_N_ZIPS).index.tolist()
print("Top postal codes tested:", top_zips)

df_topzip = df_clean[df_clean['PostalCode'].isin(top_zips)].copy()
zip_table = df_topzip.groupby('PostalCode').apply(kpi_summary).sort_values('n', ascending=False)
display(zip_table)


cont_zip = pd.crosstab(df_topzip['PostalCode'], df_topzip['has_claim'])
chi2_zip, p_chi_zip, _, _ = stats.chi2_contingency(cont_zip)
print("Top-zips claim freq chi-square: chi2=%.3f, p=%.5f" % (chi2_zip, p_chi_zip))


sev_groups_zip = [g['severity_per_claim'].dropna().values 
                  for _, g in df_topzip.groupby('PostalCode')]
sev_groups_zip = [g for g in sev_groups_zip if len(g) > 0]

if len(sev_groups_zip) > 1:
    all_vals = np.concatenate(sev_groups_zip)
    
    if np.unique(all_vals).size == 1:
        print("Top-zips severity Kruskal-Wallis: cannot run (all values identical).")
    else:
        kw_zip_s, p_zip_s = stats.kruskal(*sev_groups_zip)
        print("Top-zips severity Kruskal-Wallis: H=%.3f, p=%.5f" % (kw_zip_s, p_zip_s))
else:
    print("Not enough severity data across top zips.")


Top postal codes tested: [2196.0, 7785.0, 2000.0, 7460.0, 2744.0, 7780.0, 7945.0, 1496.0, 7888.0, 7405.0]


C:\Users\hp\AppData\Local\Temp\ipykernel_12536\3959912211.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zip_table = df_topzip.groupby('PostalCode').apply(kpi_summary).sort_values('n', ascending=False)


,n,claim_freq,severity,mean_margin,loss_ratio
PostalCode,,,,,
2196.0,129.0,0.007752,850.0,6330.823372,0.001040
7785.0,11.0,0.000000,NaN,3393.977493,0.000000
2000.0,7.0,0.142857,850.0,27445.043085,0.004405
7945.0,5.0,0.000000,NaN,1349.525924,0.000000
7460.0,5.0,0.000000,NaN,8553.026001,0.000000
2744.0,5.0,0.000000,NaN,12996.403177,0.000000
7780.0,5.0,0.000000,NaN,3067.218119,0.000000
1496.0,4.0,0.000000,NaN,12014.795895,0.000000
7405.0,4.0,0.000000,NaN,10088.437454,0.000000


Top-zips claim freq chi-square: chi2=11.609, p=0.23625
Top-zips severity Kruskal-Wallis: cannot run (all values identical).


In [24]:
margin_groups_zip = [g['margin'].dropna().values for name,g in df_topzip.groupby('PostalCode')]
margin_groups_zip = [g for g in margin_groups_zip if len(g)>0]
if len(margin_groups_zip) > 1:
    kw_zip_m, p_zip_m = stats.kruskal(*margin_groups_zip)
    print("Top-zips margin Kruskal-Wallis: H=%.3f, p=%.5f" % (kw_zip_m, p_zip_m))
else:
    print("Not enough margin data across top zips.")


Top-zips margin Kruskal-Wallis: H=33.308, p=0.00012


In [26]:
print("Gender counts:\n", df_clean['gender_clean'].value_counts())


counts = df_clean.groupby('gender_clean')['has_claim'].sum()
ns = df_clean.groupby('gender_clean')['has_claim'].count()

if len(counts) == 2:
    z_stat, p_z = proportions_ztest(counts.values, ns.values, alternative='two-sided')

    proportions = counts.values / ns.values
    diff = proportions[0] - proportions[1]

    print(f"Two-proportion z-test: z={z_stat:.3f}, p={p_z:.5f}, "
          f"prop_diff={diff:.4f} (group order: {list(counts.index)})")

else:
    print("Gender groups != 2, cannot run two-proportion z-test. Groups:", list(counts.index))




female_sev = df_clean.loc[df_clean['gender_clean'] == 'F', 'severity_per_claim'].dropna()
male_sev   = df_clean.loc[df_clean['gender_clean'] == 'M', 'severity_per_claim'].dropna()

if len(female_sev) > 1 and len(male_sev) > 1:
    t_s, p_s = stats.ttest_ind(female_sev, male_sev, equal_var=False, nan_policy='omit')
    print(
        f"Severity Welch t-test: t={t_s:.3f}, p={p_s:.5f}, "
        f"meanF={female_sev.mean():.2f}, meanM={male_sev.mean():.2f}, "
        f"mean_diff={female_sev.mean()-male_sev.mean():.2f}"
    )
else:
    print("Not enough severity data in one of the gender groups.")



female_m = df_clean.loc[df_clean['gender_clean'] == 'F', 'margin'].dropna()
male_m   = df_clean.loc[df_clean['gender_clean'] == 'M', 'margin'].dropna()

if len(female_m) > 1 and len(male_m) > 1:
    t_m, p_m = stats.ttest_ind(female_m, male_m, equal_var=False, nan_policy='omit')
    print(
        f"Margin Welch t-test: t={t_m:.3f}, p={p_m:.5f}, "
        f"meanF={female_m.mean():.2f}, meanM={male_m.mean():.2f}, "
        f"mean_diff={female_m.mean()-male_m.mean():.2f}"
    )
else:
    print("Not enough margin data in one of the gender groups.")


Gender counts:
 gender_clean
M          149
UNKNOWN    128
F           23
Name: count, dtype: int64
Gender groups != 2, cannot run two-proportion z-test. Groups: ['F', 'M', 'UNKNOWN']
Not enough severity data in one of the gender groups.
Margin Welch t-test: t=0.593, p=0.55807, meanF=13041.00, meanM=10622.98, mean_diff=2418.02


In [27]:
zipA = top_zips[0] if len(top_zips)>0 else None
zipB = top_zips[1] if len(top_zips)>1 else None
print("Zip A:", zipA, "Zip B:", zipB)

if zipA and zipB:
    ga = df_clean[df_clean['PostalCode']==zipA]
    gb = df_clean[df_clean['PostalCode']==zipB]
    print("Counts:", len(ga), len(gb))
    numeric_covs = [c for c in ['CustomValueEstimate','CapitalOutstanding','SumInsured','Cylinders','kilowatts','NumberOfDoors'] if c in df_clean.columns]
    for c in numeric_covs:
        if ga[c].dropna().size>1 and gb[c].dropna().size>1:
            t, p = stats.ttest_ind(ga[c].dropna(), gb[c].dropna(), equal_var=False, nan_policy='omit')
            print(f"Numeric covariate {c}: p={p:.4f} (t-test)")
    cat_covs = [c for c in ['CoverCategory','CoverType','CoverGroup','Product','Section','StatutoryClass','StatutoryRiskType','MaritalStatus','Citizenship'] if c in df_clean.columns]
    for c in cat_covs:
        tab = pd.crosstab(df_clean['PostalCode'], df_clean[c])
        if zipA in tab.index and zipB in tab.index and tab.loc[[zipA,zipB]].sum().sum()>0:
            chi2_c, p_c, _, _ = stats.chi2_contingency(tab.loc[[zipA,zipB]])
            print(f"Categorical covariate {c}: p={p_c:.4f} (chi-square)")
else:
    print("Not enough top zips to run balance check example.")


Zip A: 2196.0 Zip B: 7785.0
Counts: 129 11


In [28]:
prov_table.to_csv("province_kpi_summary.csv", index=True)
zip_table.to_csv("zip_kpi_summary_topN.csv", index=True)
df_clean.to_csv("df_clean_policy_level.csv", index=True)
print("Saved summaries: province_kpi_summary.csv, zip_kpi_summary_topN.csv, df_clean_policy_level.csv")

plt.figure(figsize=(8,5))
prov_table['claim_freq'].sort_values().plot(kind='barh')
plt.title('Claim Frequency by Province')
plt.tight_layout()
plt.savefig("claim_freq_by_province.png", dpi=150)
plt.close()

plt.figure(figsize=(8,5))
prov_table['loss_ratio'].sort_values().plot(kind='barh')
plt.title('Loss Ratio by Province')
plt.tight_layout()
plt.savefig("loss_ratio_by_province.png", dpi=150)
plt.close()
print("Saved plots.")


Saved summaries: province_kpi_summary.csv, zip_kpi_summary_topN.csv, df_clean_policy_level.csv
Saved plots.
